In [59]:
# import cudf
# from cuml import RandomForestClassifier as cuRF

from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer 

import numpy as np
import pandas as pd
import re
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
import string

from IPython.display import display, HTML

In [14]:
HASHTAG_REGEXP = "#\w+"
LINK_REGEXP = r"https?://[\w./-]+"
HANDLE_REGEXP = "@\S+"
STOPWORDS = stopwords.words('english')

In [3]:
def clean_location(row, location_col='location'):
    if row[location_col] is np.nan or not row[location_col].strip().replace("\'|#|@", ""):
        return "N/A"
    ret_val = re.sub("\W+", ' ', row[location_col].lower())
    ret_val = re.sub("[\s\d]+", ' ', ret_val.strip()).strip()
    if len(ret_val) == 0:
        return "N/A"
    else:
        return ret_val

def lemmatize_sentence(text, tokenizer, lemmatizer):
    tokens_pos_tag = pos_tag(tokenizer.tokenize(text))
    tag_dict = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV,
    }
    tokens_wordnet = [
        (word, tag_dict.get(pos[0], wordnet.NOUN)) for word, pos in tokens_pos_tag
    ]
    lem_tokens = [lemmatizer.lemmatize(word, pos) for word, pos in tokens_wordnet]
    return lem_tokens

In [62]:
def add_features(df, tokenizer, lemmatizer, text_col="text", keyword_col="keyword"):
    ret_df = df.copy()
    return ret_df.assign(
        keyword=lambda x: ret_df[keyword_col].fillna("N/A"),
        hashtag_in_text=ret_df[text_col].str.contains(HASHTAG_REGEXP),
        hashtags=ret_df[text_col].apply(lambda x: re.findall(HASHTAG_REGEXP, x)),
        link_in_text=ret_df[text_col].str.contains(LINK_REGEXP),
        links=ret_df[text_col].apply(lambda x: re.findall(LINK_REGEXP, x)),
        mention_in_text=ret_df[text_col].str.contains(HANDLE_REGEXP),
        mentions=ret_df[text_col].apply(lambda x: re.findall(HANDLE_REGEXP, x)),
        location=ret_df.apply(clean_location, location_col="location", axis=1),
        tokens=ret_df[text_col].apply(tokenizer.tokenize),
        lem_tokens=ret_df[text_col].apply(
            lemmatize_sentence, tokenizer=tokenizer, lemmatizer=lemmatizer
        ),
        lem_text=lambda x: x["lem_tokens"].apply(" ".join),
        word_count=lambda x: x["tokens"].apply(len),
        unique_tokens=lambda x: x["tokens"].apply(set),
        unique_word_count=lambda x: x["unique_tokens"].apply(len),
        stop_word_count=lambda x: x["tokens"].apply(
            lambda y: len([token for token in y if token in STOPWORDS])
        ),
        link_count=lambda x: x["links"].apply(len),
        hashtag_count=lambda x: x["hashtags"].apply(len),
        mention_count=lambda x: x["mentions"].apply(len),
        char_count=lambda x: x["tokens"].apply(
            lambda y: np.sum([len(token) for token in y])
        ),
        mean_word_length=lambda x: x["char_count"] / x["word_count"],
        punctuation_count=lambda x: x["tokens"].apply(
            lambda y: len([token for token in y if token in string.punctuation])
        ),
    )


def reduce_locations(df, location_col="location", id_col="id"):
    # Handle location categories that only appear a single time by replacing them with N/A
    ret_df = df.copy()

    single_location_list = (
        ret_df.groupby(location_col, as_index=False)
        .agg({id_col: "count"})
        .rename(columns={id_col: "row_count"})
        .query("row_count == 1")[location_col]
        .tolist()
    )

    return ret_df.assign(
        location=ret_df[location_col].where(
            ~ret_df[location_col].isin(single_location_list), other="N/A"
        )
    )


# train_df = train_df.assign(location = train_df["location_orig"].where(
#     ~train_df["location_orig"].isin(single_location_list), other="N/A"
# ))

In [63]:
wnl = WordNetLemmatizer()
tknzr = TweetTokenizer(reduce_len=True)

train_df = (pd.read_csv("./train.csv")
           .pipe(add_features, tokenizer=tknzr, lemmatizer=wnl)
           .pipe(reduce_locations))

### Features to Add
- word_count number of words in text
- unique_word_count number of unique words in text
- stop_word_count number of stop words in text
- url_count number of urls in text
- mean_word_length average character count in words
- char_count number of characters in text
- punctuation_count number of punctuations in text
- hashtag_count number of hashtags (#) in text
- mention_count number of mentions (@) in text

,id,keyword,location,text,target,hashtag_in_text,hashtags,link_in_text,links,mention_in_text,...,word_count,unique_tokens,unique_word_count,stop_word_count,link_count,hashtag_count,mention_count,char_count,mean_word_length,punctuation_count
0,1,N/A,N/A,Our Deeds are the Reason of this #earthquake M...,1,True,[#earthquake],False,[],False,...,13,"{of, Forgive, this, May, the, Deeds, Our, #ear...",13,5,0,1,0,57,4.384615,0
1,4,N/A,N/A,Forest fire near La Ronge Sask. Canada,1,False,[],False,[],False,...,8,"{Sask, Canada, Forest, La, near, ., Ronge, fire}",8,0,0,0,0,32,4.000000,1
2,5,N/A,N/A,All residents asked to 'shelter in place' are ...,1,False,[],False,[],False,...,25,"{expected, officers, ., to, ', in, residents, ...",20,9,0,0,0,112,4.480000,3
3,6,N/A,N/A,"13,000 people receive #wildfires evacuation or...",1,True,[#wildfires],False,[],False,...,8,"{orders, receive, #wildfires, California, in, ...",8,1,0,1,0,57,7.125000,0
4,7,N/A,N/A,Just got sent this photo from Ruby #Alaska as ...,1,True,"[#Alaska, #wildfires]",False,[],False,...,16,"{school, as, sent, into, this, #wildfires, fro...",15,6,0,2,0,72,4.500000,0


In [29]:
train_df['tokens'].apply(lambda x: len([token for token in x if token in string.punctuation]))

0       0
1       1
2       3
3       0
4       0
       ..
7608    0
7609    2
7610    5
7611    2
7612    2
Name: tokens, Length: 7613, dtype: int64

In [58]:
train_df.assign(
    word_count=lambda x: x["tokens"].apply(len),
    unique_tokens=lambda x: x["tokens"].apply(set),
    unique_word_count=lambda x: x["unique_tokens"].apply(len),
    stop_word_count=lambda x: x["tokens"].apply(
        lambda y: len([token for token in y if token in STOPWORDS])
    ),
    link_count=lambda x: x["links"].apply(len),
    hashtag_count=lambda x: x["hashtags"].apply(len),
    mention_count=lambda x: x["mentions"].apply(len),
    char_count=lambda x: x["tokens"].apply(lambda y: np.sum([len(token) for token in y])),
    mean_word_length=lambda x: x["char_count"] / x["word_count"],
    punctuation_count=lambda x: x["tokens"].apply(
        lambda y: len([token for token in y if token in string.punctuation])
    ),
)

,id,keyword,location,text,target,hashtag_in_text,hashtags,link_in_text,links,mention_in_text,...,word_count,unique_tokens,unique_word_count,stop_word_count,link_count,hashtag_count,mention_count,char_count,mean_word_length,punctuation_count
0,1,N/A,N/A,Our Deeds are the Reason of this #earthquake M...,1,True,[#earthquake],False,[],False,...,13,"{of, Forgive, this, May, the, Deeds, Our, #ear...",13,5,0,1,0,57,4.384615,0
1,4,N/A,N/A,Forest fire near La Ronge Sask. Canada,1,False,[],False,[],False,...,8,"{Sask, Canada, Forest, La, near, ., Ronge, fire}",8,0,0,0,0,32,4.000000,1
2,5,N/A,N/A,All residents asked to 'shelter in place' are ...,1,False,[],False,[],False,...,25,"{expected, officers, ., to, ', in, residents, ...",20,9,0,0,0,112,4.480000,3
3,6,N/A,N/A,"13,000 people receive #wildfires evacuation or...",1,True,[#wildfires],False,[],False,...,8,"{orders, receive, #wildfires, California, in, ...",8,1,0,1,0,57,7.125000,0
4,7,N/A,N/A,Just got sent this photo from Ruby #Alaska as ...,1,True,"[#Alaska, #wildfires]",False,[],False,...,16,"{school, as, sent, into, this, #wildfires, fro...",15,6,0,2,0,72,4.500000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,10869,N/A,N/A,Two giant cranes holding a bridge collapse int...,1,False,[],True,[http://t.co/STfMbbZFB5],False,...,11,"{giant, into, homes, collapse, Two, nearby, br...",11,2,1,0,0,73,6.636364,0
7609,10870,N/A,N/A,@aria_ahrary @TheTawniest The out of control w...,1,False,[],False,[],True,...,22,"{of, state, even, troubling, fires, California...",18,7,0,0,2,106,4.818182,2
7610,10871,N/A,N/A,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,False,[],True,[http://t.co/zDtoyd8EbJ],False,...,15,"{Volcano, of, [, 5km, M1, UTC, ., 94, ], http:...",14,1,1,0,0,58,3.866667,5
7611,10872,N/A,N/A,Police investigating after an e-bike collided ...,1,False,[],False,[],False,...,21,"{E-bike, injuries, ., in, an, with, collided, ...",20,5,0,0,0,119,5.666667,2


In [4]:
# from nltk.tag import StanfordNERTagger, StanfordPOSTagger

- Function to take the sentence and output the lemmatized list
    - Input is text
    - Next tokenize and get pos_tag
    - Use the pos_tage with token as input to the lemmatizer
    

postag = StanfordPOSTagger('english-bidirectional-distsim.tagger',
                           '/home/sjustice/My_code/stanford-tagger-4.0.0/models/english-bidirectional-distsim.tagger')

nert = StanfordNERTagger('/home/sjustice/My_code/stanford-tagger-4.0.0/models/')

In [6]:
# from nltk import download
# download('averaged_perceptron_tagger')
# download('wordnet')

- There are a lot of location categories that only appear once
    - 2818 locations only appear once
    - Some of them are in lowercase
    - Convert the location to lower and remove punctuation
    - Maybe remove those or ones with less than 5 occurances and replace them with None?

- Replace all locations that appear only once with N/A

- Perform a train/test split and then use the stemmer and the vectorizer on the train set
- Need to be able to align the features from the train set so that they are the same in the test set
    - Only include the ones in the train set - No new features in the test set
    
- Perform lematization on the text. 

In [85]:
X_train, X_test, y_train, y_test = train_test_split(
    train_df[
        [
            "keyword",
            "location",
            "text",
            "hashtag_in_text",
            "link_in_text",
            "handle_in_text",
#             "tokens",
#             "lem_tokens",
            "lem_text",
        ]
    ],
    train_df["target"],
    random_state=42,
    stratify=train_df["target"],
    test_size=0.2,
)

In [13]:
cat_feature_list =['keyword', 'location', 'hash_in_text', 'handle_in_text']

# Use a column transformer instead of the FeatureUnion since it accomplishes the same thing without
# needing a custom class

pipe = Pipeline(
    [
        (
            "text_transform",
            ColumnTransformer([("ohe", OneHotEncoder(handle_unknown='ignore'), cat_feature_list),
                                  ("tfidf", TfidfVectorizer(), 'lem_text')]),
        ),
#         ("cudf_convert", CudfConversion()),
        ("rf", RandomForestClassifier(random_state=42, n_jobs=1) )
    ]
)

- Things to check for in the text
    - @s - tweets directed at other users
    - #s - hashtags
    - retweets - urls
    - Name entity
    - Allcaps words??? - Remove them or keep them?
    - Remove the ats, hashtags, and urls from the text before putting it through a tfidf vectorizer
    - A lot of repeated location categories - Need to handle instances like 'Chicago, IL' and 'Chicago,IL'

In [14]:
param_grid = {
    "rf__n_estimators": range(3000, 10001, 500),
    "rf__max_depth": [None, 10, 20, 40],
    "rf__min_samples_split": [2, 4, 6],
    "rf__min_samples_leaf": [1, 2, 3],
    "rf__max_features": ["auto", "log2", 20, 40, 60, 100],
    "text_transform__tfidf__ngram_range": [(1,1), (1,2), (1,3), (2,3)],
    "text_transform__tfidf__max_df": [0.9, 0.95, 0.99],
    "text_transform__tfidf__min_df": [2, 5, 10]
}

In [17]:
rcv = RandomizedSearchCV(estimator=pipe, param_distributions=param_grid, n_iter=30,
                         scoring='f1', cv=3, random_state=42, verbose=2, n_jobs=7)

In [18]:
rcv.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:  5.5min
[Parallel(n_jobs=7)]: Done  90 out of  90 | elapsed: 25.8min finished


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('text_transform',
                                              ColumnTransformer(transformers=[('ohe',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               ['keyword',
                                                                                'location',
                                                                                'hash_in_text',
                                                                                'handle_in_text']),
                                                                              ('tfidf',
                                                                               TfidfVectorizer(),
                                                                               'lem_text')])),
                                             (

In [20]:
rcv.best_params_

{'text_transform__tfidf__ngram_range': (1, 1),
 'text_transform__tfidf__min_df': 10,
 'text_transform__tfidf__max_df': 0.99,
 'rf__n_estimators': 7000,
 'rf__min_samples_split': 4,
 'rf__min_samples_leaf': 1,
 'rf__max_features': 'auto',
 'rf__max_depth': None}

In [81]:
best_pipe = rcv.best_estimator_

In [80]:
rcv.score(X_test, y_test)

0.7210300429184551

In [79]:
import pickle

with open('rf_model_with_pipe.pkl', 'wb') as f:
    pickle.dump(rcv, f)

# # and later you can load it
# with open('filename.pkl', 'rb') as f:
#     clf = pickle.load(f)

In [79]:
test_df = (pd.read_csv("./test.csv")
           .pipe(add_features, tokenizer=tknzr, lemmatizer=wnl)
           .pipe(reduce_locations))

In [88]:
X_train.head()

,keyword,location,text,hashtag_in_text,link_in_text,handle_in_text,lem_text
6234,snowstorm,south usa,Sassy city girl country hunk stranded in Smoky...,True,True,False,Sassy city girl country hunk strand in Smoky M...
326,armageddon,worldwide,God's Kingdom (Heavenly Gov't) will rule over ...,False,True,False,God's Kingdom ( Heavenly Gov't ) will rule ove...
997,body%20bagging,N/A,Mopheme and Bigstar Johnson are a problem in t...,True,False,False,Mopheme and Bigstar Johnson be a problem in th...
7269,whirlwind,N/A,@VixMeldrew sounds like a whirlwind life!,False,False,True,@VixMeldrew sound like a whirlwind life !
2189,debris,nigeria,Malaysia confirms plane debris washed up on Re...,False,True,False,Malaysia confirm plane debris wash up on Reuni...


In [89]:
test_df[[
            "keyword",
            "location",
            "text",
            "hashtag_in_text",
            "link_in_text",
            "handle_in_text",
#             "tokens",
#             "lem_tokens",
            "lem_text",
        ]].head()

,keyword,location,text,hashtag_in_text,link_in_text,handle_in_text,lem_text
0,N/A,N/A,Just happened a terrible car crash,False,False,False,Just happen a terrible car crash
1,N/A,N/A,"Heard about #earthquake is different cities, s...",True,False,False,"Heard about #earthquake be different city , st..."
2,N/A,N/A,"there is a forest fire at spot pond, geese are...",False,False,False,"there be a forest fire at spot pond , geese be..."
3,N/A,N/A,Apocalypse lighting. #Spokane #wildfires,True,False,False,Apocalypse light . #Spokane #wildfires
4,N/A,N/A,Typhoon Soudelor kills 28 in China and Taiwan,False,False,False,Typhoon Soudelor kill 28 in China and Taiwan


In [90]:
best_pipe.predict(test_df[[
            "keyword",
            "location",
            "text",
            "hashtag_in_text",
            "link_in_text",
            "handle_in_text",
#             "tokens",
#             "lem_tokens",
            "lem_text",
        ]].rename(columns={"hashtag_in_text": "hash_in_text"}))

array([0, 1, 1, ..., 1, 1, 0])

In [91]:
test_df = test_df.assign(target=best_pipe.predict(test_df[[
            "keyword",
            "location",
            "text",
            "hashtag_in_text",
            "link_in_text",
            "handle_in_text",
#             "tokens",
#             "lem_tokens",
            "lem_text",
        ]].rename(columns={"hashtag_in_text": "hash_in_text"})))

In [94]:
test_df[['id', 'target']].to_csv("submission_0706.csv", index=False)